In [2]:
from pathlib import Path
import os

# 경로 설정
SRC = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250902_data_origin")

# 이미지 확장자
img_exts = {".jpg", ".jpeg", ".png", ".bmp"}

# 정렬된 이미지 리스트
images = sorted([p for p in SRC.rglob("*") if p.suffix.lower() in img_exts])

print(f"[INFO] Found {len(images)} images")

# === 이름 바꾸기 ===
prefix = "250902"  # 원하는 접두사
start_idx = 1

for i, img in enumerate(images, start=start_idx):
    new_name = f"{prefix}_{i:05d}{img.suffix.lower()}"  # 5자리 번호
    new_path = img.parent / new_name
    os.rename(img, new_path)

print("[DONE] 파일 이름 변경 완료 ✅")


[INFO] Found 10080 images
[DONE] 파일 이름 변경 완료 ✅


In [10]:
# ============================================================
# YOLOv8 자동 라벨링 + 통계 출력
# Src : /home/dw/ws_job_msislab/Golf_Project/data/20250902_data_start_label
# Out : 같은 폴더에 동일 stem의 .txt 파일 (YOLO xywh normalized)
# Model : good_data_2_aug_train/weights/best.pt
# Classes : 0=Divot, 1=Fixed_Divot, 2=Dieased_grass
# ============================================================

from ultralytics import YOLO
from pathlib import Path

# 경로 설정
SRC = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250902_data_start_label")
WEIGHT = "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/good_data_2_aug_train/weights/best.pt"

# 모델 로드
model = YOLO(WEIGHT)

# 이미지 파일 모으기
img_exts = {".jpg", ".jpeg", ".png", ".bmp"}
images = [p for p in SRC.rglob("*") if p.suffix.lower() in img_exts]

print(f"[INFO] Found {len(images)} images")

# === 집계용 변수 ===
total_images = len(images)
labeled_images = 0
total_labels = 0

# 라벨 저장 함수 (박스가 없으면 파일 자체를 만들지 않음)
def save_yolo_labels(result, save_dir):
    global labeled_images, total_labels
    stem = Path(result.path).stem
    label_path = Path(save_dir) / f"{stem}.txt"

    lines = []
    for box in result.boxes:
        cls_id = int(box.cls[0].item())
        conf   = float(box.conf[0].item())
        cx, cy, w, h = box.xywhn[0].tolist()  # [cx, cy, w, h], normalized

        # 원하는 confidence threshold 적용
        if conf >= 0.75:
            lines.append(f"{cls_id} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")

    if lines:  # 박스가 하나라도 있을 때만 파일 생성
        with open(label_path, "w") as f:
            f.writelines(lines)
        labeled_images += 1
        total_labels += len(lines)

# 추론 및 저장
for img in images:
    results = model(img, imgsz=896, conf=0.7, device=0, half=True, verbose=False)
    for r in results:
        save_yolo_labels(r, img.parent)  # 같은 폴더에 저장

# === 결과 출력 ===
print("====================================")
print(f"[RESULT] 총 이미지 수         : {total_images}")
print(f"[RESULT] 라벨링된 이미지 수   : {labeled_images}")
print(f"[RESULT] 생성된 라벨(박스) 수 : {total_labels}")
print("====================================")
print("[DONE] 자동 라벨링 완료 ✅")


[INFO] Found 10080 images
[RESULT] 총 이미지 수         : 10080
[RESULT] 라벨링된 이미지 수   : 4263
[RESULT] 생성된 라벨(박스) 수 : 6168
[DONE] 자동 라벨링 완료 ✅
